# <font color='violet'> Complete Language Processing, Continue Deeper EDA
    
Using prescription drug review data analyzed and parsed here: https://github.com/fractaldatalearning/psychedelic_efficacy/blob/main/notebooks/3-kl-studies-early-eda-parse.ipynb

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../data/interim/studies_early_parsing.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31559 entries, 0 to 31558
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     31559 non-null  int64  
 1   rating         31559 non-null  float64
 2   condition      31559 non-null  object 
 3   review         31559 non-null  object 
 4   date           31451 non-null  object 
 5   drug0          31559 non-null  object 
 6   drug1          18992 non-null  object 
 7   drug2          32 non-null     object 
 8   drug3          23 non-null     object 
 9   drug4          12 non-null     object 
 10  drug5          11 non-null     object 
 11  drug6          7 non-null      object 
 12  drug7          5 non-null      object 
 13  drug8          3 non-null      object 
 14  drug9          2 non-null      object 
 15  drug10         2 non-null      object 
 16  drug11         2 non-null      object 
 17  drug12         2 non-null      object 
 18  drug13

In [6]:
# Delete unnamed column and columns I'd used for eda previously but won't need here.
df = df.drop(columns = ['Unnamed: 0', 'ratings_count', 'count_by_date'])
df.head(3)

,rating,condition,review,date,drug0,drug1,drug2,drug3,drug4,drug5,drug6,drug7,drug8,drug9,drug10,drug11,drug12,drug13,drug14,drug15
0,9.0,add,I had began taking 20mg of Vyvanse for three m...,NaN,vyvanse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,add,Switched from Adderall to Dexedrine to compare...,NaN,dextroamphetamine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.0,adhd,I have only been on Vyvanse for 2 weeks I sta...,NaN,vyvanse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<font color='violet'> Lemmatize text and do further NLP & EDA with review column. 
    
Resources with tips for effective EDA visualization with NLP:

https://medium.com/plotly/nlp-visualisations-for-clear-immediate-insights-into-text-data-and-outputs-9ebfab168d5b
    
https://www.numpyninja.com/post/nlp-text-data-visualization
    
https://www.kaggle.com/code/sainathkrothapalli/nlp-visualisation-guide
    
https://medium.com/acing-ai/visualizations-in-natural-language-processing-2ca60dd34ce
    
https://towardsdatascience.com/a-complete-exploratory-data-analysis-and-visualization-for-text-data-29fb1b96fb6a
    
https://towardsdatascience.com/getting-started-with-text-nlp-visualization-9dcb54bc91dd
    
https://www.kaggle.com/code/mitramir5/nlp-visualization-eda-glove
    
https://medium.com/analytics-vidhya/how-to-begin-performing-eda-on-nlp-ffdef92bedf6
    
https://inside-machinelearning.com/en/eda-nlp/
    
https://towardsdatascience.com/fundamental-eda-techniques-for-nlp-f81a93696a75
    
https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools
    
https://www.kdnuggets.com/2019/05/complete-exploratory-data-analysis-visualization-text-data.html
    
